# Graphene

Exercise 1:
Repeat the explorations done with a square lattice: try to answer the following questions and justify your
sentences with graphs. To answer some of the questions you may have to explore the literature on graphene
(with no need to go very deep, a good review or a seminal paper shall bring you all the informations).

1) What is the upper limit of E which gives reasonable results? We will in the following restrict our study
to energies below this threshold.

2) What happens for E < 0? How does it compare to the case of a square lattice? Which property of
graphene does it underline?

3) How would you qualify the way transmission evolves with energy in the low density limit (the average evolution, not its
discretization)? How does it compare to the square lattice? Which property of graphene does it underline?

4) Focus on the very low energy limit of the dispersion relation, where discrete steps can be observed.
Comment their spacing and compare with the square lattice. Which property of graphene does it underline?

In [1]:
import kwant
from kwant.digest import uniform    # a (deterministic) pseudorandom number generator
import numpy as np
import math
from math import atan2, pi, sqrt, cos, sin
from cmath import exp
from types import SimpleNamespace
import matplotlib
from matplotlib import pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 5,5

In [2]:
h = 6.626069e-34    # Planck constant (Js)
e = 1.602e-19      # electron charge (C)
c = 299792458       # Light celerity (m/s)
kB = 1.3806503e-23  # Boltzmann constant (kg m2/Ks2)
m_e = 9.1e-31       # electron mass (kg)

In [3]:
def hopping(site_i, site_j):
    """
    Definition of the hopping parameter by including the magnetic field (parameter phi)
    
    """
    
    xi, yi = site_i.pos
    xj, yj = site_j.pos
    return -t * exp(-0.5j * B * (xi - xj) * (yi + yj))

In [4]:
def make_system(a=1,W=20,L=20,t=1.0):
    graphene = kwant.lattice.honeycomb(a,norbs=1)
    a, b = graphene.sublattices
    sys = kwant.Builder()
 
    def rectangular(pos):
        x, y = pos
        return abs(x) < L/2 and abs(y) < W/2
    def lead_shape(pos):
        x, y = pos
        return -W/2 < y < W/2
  
    sys[graphene.shape(rectangular, (0,0))] = onsite
    hoppings = (((0, 0), a, b), ((0, 1), a, b), ((-1, 1), a, b))
    sys[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings]] = hopping

    sym0 = kwant.TranslationalSymmetry(graphene.vec((-1, 0)))
    lead0 = kwant.Builder(sym0)
    lead0[graphene.shape(lead_shape, (0, 0))] = 0
    lead0[graphene.neighbors()]= hopping
    sys.attach_lead(lead0)
    
    sym1 = kwant.TranslationalSymmetry(graphene.vec((1, 0)))
    lead1 = kwant.Builder(sym1)
    lead1[graphene.shape(lead_shape, (0, 0))] = pot
    lead1[graphene.neighbors()]= hopping
    sys.attach_lead(lead1)

    return sys

In [5]:
a = 1
W = 30
L = 40
t = 1
Ef = 1
B = 1e-2

In [6]:
a = 1
W = 30
L = 20
t = 1
Ef = 1
B_eff = 0.1
sys = make_system(a,W,L,t)

kwant.plot(sys)

sys = make_system(a,W,L,t)
kwant.plot(sys)
# Finalize the system.
graph = sys.finalized()
kwant.plotter.bands(graph.leads[0])

NameError: name 'onsite' is not defined

In [ ]:
wfs = kwant.wave_function(graph, energy=Ef) # to obtain the wave functions of the system 

J0 = kwant.operator.Current(graph)
wf_left = wfs(0)
current = sum(J0(p) for p in wf_left)

kwant.plotter.current(graph, current, cmap='viridis')

In [ ]:
params = SimpleNamespace(phi=phi)#, Vg=Vg, l_pot=l_pot, w_pot=w_pot, R=R, shape=shape)

In [ ]:
""" The wavelength must be (much more) larger than the lattice parameter """
lambda_F = 3 * pi * a * t / Ef # computation of the wavelength in graphene
print('Fermi wavelength: ', lambda_F)
if lambda_F/a < 2:
    print('Warning : lambda_F must be larger than the lattice parameter a')

In [ ]:
wfs = kwant.wave_function(sys, energy=Ef) # to obtain the wave functions of the system 

J0 = kwant.operator.Current(sys)
wf_left = wfs(0)
current = sum(J0(p) for p in wf_left)

kwant.plotter.current(sys, current, cmap='viridis')

In [ ]:
N = 20 # number of magnetic field values
B_eff_max = 0.2 # higher magnetic field
B_effs = np.linspace(0, B_eff_max, N) # vector of the magnetic fields

G = np.zeros([N,1])

for i,B_eff in enumerate(B_effs):
    params = SimpleNamespace(B_eff=B_eff)
    smatrix = kwant.smatrix(sys, energy = Ef) # transmission matrix (here this)
    T = smatrix.transmission(1, 0) # transmission value obtained from the left lead towards the right lead
    G[i] = T
    
plt.plot(B_effs,G)

plt.xlabel('Magnetic field (T)')
plt.ylabel('Transmission (2e²/h)')

plt.show()

In [ ]:
pyplot.plot(E,Gr)

pyplot.xlabel('Energy (t)', fontsize=18)
pyplot.ylabel('Transmission', fontsize=18)

Answers:

1) For energies above t (t=1 here), the conductance starts to decrease whereas we still increase the carrier density, which is not physical. Once again, we leave tight binding approximationfor high energies.

2) For negative energies, we the conductance is non zero and re-increases. This is due to the fact that grahene has no gap, which differs from the square lattice. Graphene is often called a gapless semiconductor or semi-metal.

In [ ]:
# Let's make a larger system.
a = 5
W = 300
L = 20
t = 1
sys = make_system(a,W,L,t)
kwant.plot(sys)
graph = sys.finalized()
N = 60
E = np.linspace(-0.5,0.5,N)
Gr = np.zeros([N,1])
for i,Ef in enumerate(E):
    smatrix = kwant.smatrix(graph, energy = Ef)
    T = smatrix.transmission(1, 0)
    Gr[i] = T

In [ ]:
N = 20
B_eff_max = 0.2 # higher magnetic field
B_effs = np.linspace(0, B_eff_max, N) # vector of the magnetic fields
for B_eff in B_effs:
    print('Effective magnetic field : ', B_eff)
    phi = B_eff * a
    params = SimpleNamespace(B_eff=B_eff)
    wfs = kwant.wave_function(sys, energy=Ef)
    scattering_wf = wfs(0)  # all scattering wave functions from lead 0
    kwant.plotter.map(sys, np.sum(abs(scattering_wf)**2, axis=0));
    
    wf_left = wfs(0)
    current = sum(J0(p) for p in wf_left) # to sum over all the lead's mode
    kwant.plotter.current(sys, current, cmap='viridis')

In [ ]:
N = 50
E = np.linspace(-1.5,1.5,N)
Gr = np.zeros([N,1])
for i,Ef in enumerate(E):
    smatrix = kwant.smatrix(graph, energy = Ef)
    T = smatrix.transmission(1, 0)
    Gr[i] = T

In [ ]:
pyplot.plot(E,Gr)

pyplot.xlabel('Energy (t)', fontsize=18)
pyplot.ylabel('Transmission', fontsize=18)
pl